### Basic Functions for PGT KEP individual project

**made by Eunji Kwak**

In [1]:
###Setup###
import pandas as pd
import numpy as np
import math
import itertools

In [2]:
# Data Preprocessing
# Remove 77 sequences from whole dataset
def gen_valid_df(df):
    ci_df = pd.concat([df[df['Results'].eq('CI')], df[df['Results'].str.contains('CI,')]])
    cii_df = pd.concat([df[df['Results'].eq('CII')], df[df['Results'].str.contains('CII,')]])
    valid_df = pd.concat([ci_df, cii_df])
    return valid_df, ci_df, cii_df

In [3]:
# Remove the sequences in postion 15 to 20; Nicking enzyme recognition site + additional 'T'
def gen_partial_seq(seq):
    return seq[seq.find('GACTC')-14:seq.find('GACTC')]+seq[seq.find('GACTC')-14:seq.find('GACTC')-4]

In [6]:
# PWM based method
# Step01. Generate PWMs
def gen_pwm(seqs): # generate basic PWM
    mat1 = [[0 for col in range(24)] for row in range(4)]
    pwm = [[0 for col in range(24)] for row in range(4)]
    
    for seq in seqs:
        partial_seq = gen_partial_seq(seq)
        if partial_seq == '':
            continue
        for i in range(24):
            if partial_seq[i] == 'A':
                mat1[0][i] += 1
            if partial_seq[i] == 'C':
                mat1[1][i] += 1
            if partial_seq[i] == 'G':
                mat1[2][i] += 1
            if partial_seq[i] == 'T':
                mat1[3][i] += 1
    
    mat1 = np.array(mat1)
    
    for col in range(24):
        max_num = max(mat1.T[col])
        for row in range(4):
            pwm[row][col] = math.log(max_num / mat1[row][col])
            
    return pwm

def gen_p90_pwm(df): # generate PWM related to P90 score   
    p90_threshold = df.loc[:,'P90'].quantile(0.30)
    p90_template = df[df['P90'] <= p90_threshold]
    seqs = [seq for seq in p90_template.iloc[:,0]]

    return gen_pwm(seqs)

def gen_diff_pwm(df): # generate PWM related to Diff score    
    diff_threshold = df.loc[:,'DIFF'].quantile(0.60)
    diff_template = df[df['DIFF'] >= diff_threshold]
    seqs = [seq for seq in diff_template.iloc[:,0]]

    return gen_pwm(seqs)

# Step02. Calculate the P90/Diff score for the template by generated PWMs
def pwm_pred(pwm=[], seq=''):
    mat_1 = [[0 for col in range(24)] for row in range(4)]
    mat_2 = [[0 for col in range(24)] for row in range(4)]
    mat_score = [0 for x in range(24)]
    score = 0

    for i in range(24):
        if seq[i] == 'A':
            mat_1[0][i] = 1
        if seq[i] == 'C':
            mat_1[1][i] = 1
        if seq[i] == 'G':
            mat_1[2][i] = 1
        if seq[i] == 'T':
            mat_1[3][i] = 1
            
    for i in range(24):
        for j in range(4):
            mat_2[j][i] = mat_1[j][i]*pwm[j][i]
            # print(mat_1[j][i])
            
    for i in range(24):
        for j in range(4):
            mat_score[i] = mat_score[i] + mat_2[j][i]
    
    score = sum(mat_score)
    
    return score

# Step03. Classify either Class I or II
def pwm_class(P90_score, Diff_score):
    if Diff_score > -0.8510*P90_score+15.8764 :
        return 'Class II'
    else:
        return 'Class I'

In [5]:
# Naive Bayes based method
# Step01. Generate sequence motifs
# Step02. Classify with Naïve Bayes classification
def cmp(a, b):
    return (a > b) - (a < b)

def bayes_pred(seqs, class_name, eff_motifs):
    result = {} # position motif count matrix
    
    for motif in eff_motifs: 
        presence = []
        
        for seq in seqs:
            mer = motif.index('-')
            subseq = motif[0:mer]
            start = motif[mer+1:]
            
            if cmp(seq[int(start)-1:int(start)-1+len(subseq)],subseq)==0:
                presence.append(1)
                
            else:
                presence.append(0)  
                
        result['class'] = class_name
        result[motif] = presence
        
    return pd.DataFrame(result)